# SNS 데이터 분석 - 그룹 나누기

## 형태소 분석

---

In [ ]:
import os
import numpy as np
import pandas as pd

import nltk
from konlpy.tag import Twitter


### 사용자 설정

In [ ]:
# 컨텐츠 파일(원본파일 리스트) : 2개이상 파일 동시 등록 가능
fname_in_lst  = ['data/RawData_101.xlsx',
                 'data/RawData_102.xlsx',
                 'data/RawData_103.xlsx',
                 'data/RawData_104.xlsx',
                 'data/RawData_105.xlsx',
                 'data/RawData_106.xlsx',
                 'data/RawData_107.xlsx'
                 ]

# 형태소 분류된 컨텐츠 파일(결과파일 리스트) : 2개이상 파일 동시 등록 가능
fname_out_lst = ['data/morph_101.txt',
                 'data/morph_102.txt',
                 'data/morph_103.txt',
                 'data/morph_104.txt',
                 'data/morph_105.txt',
                 'data/morph_106.txt',
                 'data/morph_107.txt'
                ]

# 글자수가 LEN_MAX 미만은 삭제(2글자 이상부터 처리)
LEN_MAX = 2

### 형태소 분류 함수 정의

In [ ]:
def morph( lines ):
#
# Twitter tag list for Twitter.pos(): string --> list
#
# 'Noun', 'ProperNoun', 'Number', 'Verb', 'Adjective', 'Modifier', 'Adverb',
# 'Exclamation', 'Josa', 'Conjunction', 'PreEomi', 'Eomi', 'VerbPrefix',
# 'Suffix', 'Punctuation', 'CashTag', 'Others', 'Unknown', 'Alpha', 'Foreign'
# 'KoreanParticle'
#
    tw = Twitter()

    res_list = []
    for line in lines:
        lt_word = tw.pos(line, norm=True, stem=True)
        lt_morph = []
        for word in lt_word:
#            if not word[1] in ['Number', 'Punctuation', 'CashTag', 'Josa',
#                               'Alpha', 'Foreign', 'Others', 'Unknown', 'KoreanParticle']:
#            if word[1] in ['Noun', 'ProperNoun']:
            if word[1] == 'Noun' and len(word[0]) >= LEN_MAX:
                lt_morph.append(word[0])
        v_str = (" ".join(lt_morph)).strip()
        res_list.append(v_str)
    
    return res_list
    

### 형태소 분류 파일 생성 함수 정의

In [ ]:
#===========================================================================
# convert_file: xlsx file --> txt(\t) file ( morpheme analysis )
#===========================================================================
def convert_file( fout, *fins ):

    # Remove file
    if os.path.exists(fout): os.remove(fout)

    for fname in fins:
        try:
            df = pd.read_excel(fname, index_col='doc_id', keep_default_na=False, encoding='cp949')
        except:
            print("convert_file(): read_csv FAIL")
            continue

        df.sort_index(inplace=True)

        df['tmp'] = morph(list(df['content']))

        try:
            df['tmp'].to_csv(fout, mode='a', header=False, sep='\t', encoding='utf-8')
        except:
            print("convert_file(): to_csv FAIL")
            continue
    return
    

### 형태소 분류 파일 생성

In [ ]:
%%time
for fname_in, fname_out in zip(fname_in_lst,fname_out_lst):
    convert_file( fname_out, fname_in )
    print('Convert File: {} --> {}'.format(fname_in, fname_out))
    
    #
    # FDNX FIXME: For testing...
    #
    break


In [ ]:
# end of file